In [1]:
import pandas as pd
from functools import reduce
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
import dart_fss as dart
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
pd.options.display.float_format = '{:.6f}'.format

In [2]:
tdf_df = []

for year in range(2019, 2025):
    tdf = pd.read_csv(f'data/market_cap/tobin_q/tobin_q_{year}.csv', index_col = 0)
    tdf = tdf.replace(0, np.nan)
    tdf_df.append(tdf)
    
tdf_2019, tdf_2020, tdf_2021, tdf_2022, tdf_2023, tdf_2024 = tdf_df
tdf_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,0.807381,232706806150.000000,0.936429
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,0.669605,458365610600.000000,0.775513
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,0.077910,536014029600.000000,0.401320
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,0.713408,2929622067000.000000,2.061966
4,858364,BNK금융지주,2019,NaN,NaN,NaN,NaN,2496663984360.000000,NaN
...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,0.556077,102623429050.000000,0.949323
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,0.548405,210105000000.000000,0.787931
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,0.315212,76064246640.000000,0.446562
729,103176,흥국화재,2019,NaN,NaN,NaN,NaN,196903706925.000000,NaN


In [3]:
adf_df = []

for year in range(2019, 2025):
    adf = pd.read_csv(f'data/advertising/advertising_data_{year}.csv')
    adf = adf.replace(0, np.nan)
    adf_df.append(adf)
    
adf_2019, adf_2020, adf_2021, adf_2022, adf_2023, adf_2024 = adf_df
adf_2019

,corp_code,corp_name,stock_code,advertising_expenses
0,365387,AJ네트웍스,95570,4916319000.000000
1,125080,AK홀딩스,6840,63748087000.000000
2,219097,BGF,27410,NaN
3,1263022,BGF리테일,282330,2097000000.000000
4,858364,BNK금융지주,138930,4290223000.000000
...,...,...,...,...
726,111421,휴니드테크놀러지스,5870,635682000.000000
727,362238,휴비스,79980,514826000.000000
728,156488,휴스틸,5010,155567000.000000
729,103176,흥국화재,540,NaN


In [4]:
adf_lst = [adf_2019, adf_2020, adf_2021, adf_2022, adf_2023, adf_2024]

for df in adf_lst:
    df.drop(['corp_code'], axis = 1, inplace = True)
    
adf_2019

,corp_name,stock_code,advertising_expenses
0,AJ네트웍스,95570,4916319000.000000
1,AK홀딩스,6840,63748087000.000000
2,BGF,27410,NaN
3,BGF리테일,282330,2097000000.000000
4,BNK금융지주,138930,4290223000.000000
...,...,...,...
726,휴니드테크놀러지스,5870,635682000.000000
727,휴비스,79980,514826000.000000
728,휴스틸,5010,155567000.000000
729,흥국화재,540,NaN


In [5]:
fdf_df = []

for year in range(2019, 2025):
    fdf = pd.read_csv(f'data/foreign/foreign_ownership_ratio_{year}.csv', index_col = 0)
    fdf = fdf.replace(0, np.nan)
    fdf_df.append(fdf)
    
fdf_2019, fdf_2020, fdf_2021, fdf_2022, fdf_2023, fdf_2024 = fdf_df
fdf_2019

,corp_code,corp_name,stock_code,foreign_ownership_ratio
0,365387,AJ네트웍스,95570,7.950000
1,125080,AK홀딩스,6840,12.750000
2,219097,BGF,27410,9.000000
3,1263022,BGF리테일,282330,34.120000
4,858364,BNK금융지주,138930,51.700000
...,...,...,...,...
726,111421,휴니드테크놀러지스,5870,24.830000
727,362238,휴비스,79980,3.380000
728,156488,휴스틸,5010,4.200000
729,103176,흥국화재,540,2.170000


In [6]:
fdf_lst = [fdf_2019, fdf_2020, fdf_2021, fdf_2022, fdf_2023, fdf_2024]

for df in fdf_lst:
    df.drop(['corp_code', 'stock_code'], axis = 1, inplace = True)
    
fdf_2019

,corp_name,foreign_ownership_ratio
0,AJ네트웍스,7.950000
1,AK홀딩스,12.750000
2,BGF,9.000000
3,BGF리테일,34.120000
4,BNK금융지주,51.700000
...,...,...
726,휴니드테크놀러지스,24.830000
727,휴비스,3.380000
728,휴스틸,4.200000
729,흥국화재,2.170000


In [7]:
mdf_df = []

for year in range(2019, 2025):
    mdf = pd.read_csv(f'data/Major_Shareholder/Major_Shareholder_{year}.csv')
    mdf = mdf.replace(0, np.nan)
    mdf_df.append(mdf)
    
mdf_2019, mdf_2020, mdf_2021, mdf_2022, mdf_2023, mdf_2024 = mdf_df
mdf_2019

,year,corp_name,MSE,stock_value
0,2019,AJ네트웍스,50.050000,보통주
1,2019,AK홀딩스,64.890000,보통주
2,2019,BGF,69.980000,보통주
3,2019,BGF리테일,55.420000,보통주
4,2019,BNK금융지주,9.190000,보통주
...,...,...,...,...
725,2019,휴니드테크놀러지스,22.830000,의결권 있는 주식(보통주)
726,2019,휴비스,51.000000,보통주
727,2019,휴스틸,49.570000,보통주
728,2019,흥국화재,79.190000,보통주


In [8]:
mdf_lst = [mdf_2019, mdf_2020, mdf_2021, mdf_2022, mdf_2023, mdf_2024]

for df in mdf_lst:
    df.drop(['year', 'stock_value'], axis = 1, inplace = True)
    
mdf_2019

,corp_name,MSE
0,AJ네트웍스,50.050000
1,AK홀딩스,64.890000
2,BGF,69.980000
3,BGF리테일,55.420000
4,BNK금융지주,9.190000
...,...,...
725,휴니드테크놀러지스,22.830000
726,휴비스,51.000000
727,휴스틸,49.570000
728,흥국화재,79.190000


In [9]:
ndf_df = []

for year in range(2019, 2025):
    ndf = pd.read_csv(f'data/net_profit/net_profit_{year}.csv')
    ndf = ndf.replace(0, np.nan)
    ndf_df.append(ndf)
    
ndf_2019, ndf_2020, ndf_2021, ndf_2022, ndf_2023, ndf_2024 = ndf_df
ndf_2019

,corp_code,corp_name,year,net_profit
0,365387,AJ네트웍스,2019,42128594358.000000
1,125080,AK홀딩스,2019,57377832577.000000
2,219097,BGF,2019,17515291535.000000
3,1263022,BGF리테일,2019,151377053549.000000
4,858364,BNK금융지주,2019,NaN
...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,7964233137.000000
727,362238,휴비스,2019,-293323669.000000
728,156488,휴스틸,2019,-12543042035.000000
729,103176,흥국화재,2019,NaN


In [10]:
ndf_lst = [ndf_2019, ndf_2020, ndf_2021, ndf_2022, ndf_2023, ndf_2024]

for df in ndf_lst:
    df.drop(['corp_code', 'year'], axis = 1, inplace = True)
    
ndf_2019

,corp_name,net_profit
0,AJ네트웍스,42128594358.000000
1,AK홀딩스,57377832577.000000
2,BGF,17515291535.000000
3,BGF리테일,151377053549.000000
4,BNK금융지주,NaN
...,...,...
726,휴니드테크놀러지스,7964233137.000000
727,휴비스,-293323669.000000
728,휴스틸,-12543042035.000000
729,흥국화재,NaN


In [11]:
rdf_df = []

for year in range(2019, 2025):
    rdf = pd.read_csv(f'data/revenue/revenue_{year}.csv')
    rdf = rdf.replace(0, np.nan)
    rdf_df.append(rdf)
    
rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024 = rdf_df
rdf_2019

,corp_code,corp_name,year,previous_revenue,current_revenue
0,365387,AJ네트웍스,2019,1045526121935.000000,1202793723694.000000
1,125080,AK홀딩스,2019,3711183102751.000000,3759542062510.000000
2,219097,BGF,2019,220593918847.000000,240314527185.000000
3,1263022,BGF리테일,2019,5775862323457.000000,5946068219555.000000
4,858364,BNK금융지주,2019,NaN,NaN
...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,261298876464.000000,194668692113.000000
727,362238,휴비스,2019,1262515099360.000000,1029346443478.000000
728,156488,휴스틸,2019,588768601322.000000,536988709121.000000
729,103176,흥국화재,2019,NaN,NaN


In [12]:
rdf_lst = [rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024]

for df in rdf_lst:
    df.drop(['corp_code', 'year'], axis = 1, inplace = True)
    
rdf_2019

,corp_name,previous_revenue,current_revenue
0,AJ네트웍스,1045526121935.000000,1202793723694.000000
1,AK홀딩스,3711183102751.000000,3759542062510.000000
2,BGF,220593918847.000000,240314527185.000000
3,BGF리테일,5775862323457.000000,5946068219555.000000
4,BNK금융지주,NaN,NaN
...,...,...,...
726,휴니드테크놀러지스,261298876464.000000,194668692113.000000
727,휴비스,1262515099360.000000,1029346443478.000000
728,휴스틸,588768601322.000000,536988709121.000000
729,흥국화재,NaN,NaN


In [13]:
rddf_df = []

for year in range(2019, 2025):
    rddf = pd.read_csv(f'data/research/research_data_{year}_temp.csv')
    rddf = rddf.replace(0, np.nan)
    rddf_df.append(rddf)
    
rddf_2019, rddf_2020, rddf_2021, rddf_2022, rddf_2023, rddf_2024 = rddf_df
rddf_2024

,corp_code,corp_name,stock_code,research_expenses
0,365387,AJ네트웍스,95570,NaN
1,125080,AK홀딩스,6840,17281000000.000000
2,219097,BGF,27410,3282073000.000000
3,1263022,BGF리테일,282330,NaN
4,858364,BNK금융지주,138930,302120000000.000000
...,...,...,...,...
726,111421,휴니드테크놀러지스,5870,11502908000.000000
727,362238,휴비스,79980,NaN
728,156488,휴스틸,5010,334189000.000000
729,103176,흥국화재,540,24939000000.000000


In [14]:
rddf_lst = [rddf_2019, rddf_2020, rddf_2021, rddf_2022, rddf_2023, rddf_2024]

for df in rddf_lst:
    df.drop(['corp_code', 'stock_code'], axis = 1, inplace = True)
    
rddf_2019

,corp_name,research_expenses
0,AJ네트웍스,NaN
1,AK홀딩스,19303280000.000000
2,BGF,NaN
3,BGF리테일,NaN
4,BNK금융지주,NaN
...,...,...
726,휴니드테크놀러지스,647986957000.000000
727,휴비스,370747000.000000
728,휴스틸,NaN
729,흥국화재,NaN


In [15]:
ddf_df = []

for year in range(2020, 2025):
    ddf = pd.read_csv(f'data/2020~2024_finance_split/{year}_finance_split.csv')
    ddf_df.append(ddf)
    
ddf_2020, ddf_2021, ddf_2022, ddf_2023, ddf_2024 = ddf_df
ddf_2024

,기업명,종목코드,광고선전비,연구개발비
0,AJ네트웍스,95570,7.441910,NaN
1,AK홀딩스,6840,1052.976230,248.894120
2,BGF,27410,25.960000,0.020000
3,BGF리테일,282330,39.610000,45.350000
4,BYC,1460,14.171197,NaN
...,...,...,...,...
532,효성화학,298000,6.300000,190.974750
533,후성,93370,NaN,111.577600
534,휴비스,79980,NaN,40.298390
535,휴스틸,5010,0.962390,NaN


In [16]:
ddf_lst = [ddf_2020, ddf_2021, ddf_2022, ddf_2023, ddf_2024]

for df in ddf_lst:
    df.columns = ['corp_name', 'stock_code', 'advertising_expenses', 'research_expenses']

for df in ddf_lst:
    df.drop('stock_code', axis = 1, inplace = True)
    df.loc[:, 'advertising_expenses'] = df['advertising_expenses'] * 100000000
    df.loc[:, 'research_expenses'] = df['research_expenses'] * 100000000
    
ddf_2020

,corp_name,advertising_expenses,research_expenses
0,AJ네트웍스,2206927000.000000,NaN
1,AK홀딩스,65432074000.000000,19090994000.000000
2,BGF,4578000000.000000,2000000.000000
3,BGF리테일,3536000000.000000,3234000000.000000
4,BYC,1088112860.000000,NaN
...,...,...,...
526,효성화학,182000000.000000,20203162000.000000
527,후성,NaN,8481924000.000001
528,휴비스,NaN,8148941000.000001
529,휴스틸,115887000.000000,NaN


In [17]:
sdf_df = []

for year in range(2020, 2025):
    sdf = pd.read_csv(f'data/2020~2025_전체기업_매출액/{year}_revenue_final.csv')
    sdf_df.append(sdf)
    
sdf_2020, sdf_2021, sdf_2022, sdf_2023, sdf_2024 = sdf_df
sdf_2024

,기업명,종목코드,REVENUE
0,AJ네트웍스,95570,10141.563144
1,AK홀딩스,6840,44882.805328
2,BGF,27410,4340.854730
3,BGF리테일,282330,86987.574691
4,BYC,1460,1652.075689
...,...,...,...
695,휴니드테크놀러지스,5870,2308.005880
696,휴비스,79980,9394.275917
697,휴스틸,5010,7230.682711
698,흥국화재,540,33740.160000


In [18]:
sdf_lst = [sdf_2020, sdf_2021, sdf_2022, sdf_2023, sdf_2024]

for df in sdf_lst:
    df.columns = ['corp_name', 'stock_code', 'current_revenue']

for df in sdf_lst:
    df.drop('stock_code', axis = 1, inplace = True)
    df.loc[:, 'current_revenue'] = df['current_revenue'] * 100000000
    
sdf_2020

,corp_name,current_revenue
0,AJ네트웍스,871956093150.000000
1,AK홀딩스,2619977218250.000000
2,BGF,154381286310.000031
3,BGF리테일,6181269419290.000000
4,BYC,161863377010.000000
...,...,...
687,휴니드테크놀러지스,201269696580.000061
688,휴비스,920303309160.000000
689,휴스틸,365613120620.000000
690,흥국화재,4492959000000.000000


In [19]:
df_2019 = [tdf_2019, adf_2019, fdf_2019, mdf_2019, ndf_2019, rdf_2019, rddf_2019]
df_2020 = [tdf_2020, adf_2020, fdf_2020, mdf_2020, ndf_2020, rdf_2020, rddf_2020, ddf_2020, sdf_2020]
df_2021 = [tdf_2021, adf_2021, fdf_2021, mdf_2021, ndf_2021, rdf_2021, rddf_2021, ddf_2021, sdf_2021]
df_2022 = [tdf_2022, adf_2022, fdf_2022, mdf_2022, ndf_2022, rdf_2022, rddf_2022, ddf_2022, sdf_2022]
df_2023 = [tdf_2023, adf_2023, fdf_2023, mdf_2023, ndf_2023, rdf_2023, rddf_2023, ddf_2023, sdf_2023]
df_2024 = [tdf_2024, adf_2024, fdf_2024, mdf_2024, ndf_2024, rdf_2024, rddf_2024, ddf_2024, sdf_2024]

In [20]:
df_lst = [df_2019, df_2020, df_2021, df_2022, df_2023, df_2024]
total = {}

In [21]:
for year, df in zip(range(2019, 2025), df_lst):
    total[f'total_{year}'] = reduce(lambda x, y: pd.merge(x, y, on = 'corp_name', how = 'left'), df)

In [22]:
total_2019 = total['total_2019']
total_2020 = total['total_2020']
total_2021 = total['total_2021']
total_2022 = total['total_2022']
total_2023 = total['total_2023']
total_2024 = total['total_2024']

In [23]:
total_lst = [total_2019, total_2020, total_2021, total_2022, total_2023, total_2024]

# for df in total_lst:
#     df.fillna(0, inplace = True)

In [24]:
total_2019.isnull().sum()

corp_code                    0
corp_name                    0
year                         0
total_asset                 44
asset_size                  44
total_debt                  44
debt_ratio                  44
market_cap                   1
tobins_q                    44
stock_code                   0
advertising_expenses       210
foreign_ownership_ratio     12
MSE                         11
net_profit                  54
previous_revenue            61
current_revenue             61
research_expenses          292
dtype: int64

In [25]:
total_2020.head()

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q,stock_code,advertising_expenses_x,foreign_ownership_ratio,MSE,net_profit,previous_revenue,current_revenue_x,research_expenses_x,advertising_expenses_y,research_expenses_y,current_revenue_y
0,365387,AJ네트웍스,2020,1588170350000.000000,28.093604,1300162539894.000000,0.818654,216084891425,0.954713,95570,3323899000.000000,7.950000,49.700000,-3316812122.000000,1000259401598.000000,1017050375182.000000,NaN,2206927000.000000,NaN,871956093150.000000
1,125080,AK홀딩스,2020,4152091609064.000000,29.054633,2908596226989.000000,0.700514,333176159150,0.780756,6840,65432074000.000000,12.750000,64.900000,-265775058035.000000,3759542062510.000000,2619977218245.000000,19090994000.000000,65432074000.000000,19090994000.000000,2619977218250.000000
2,219097,BGF,2020,1658204361732.000000,28.136756,121105273379.000000,0.073034,468533691945,0.355589,27410,4578000000.000000,9.000000,69.720000,17880953455.000000,133387713837.000000,154381286305.000000,NaN,4578000000.000000,2000000.000000,154381286310.000031
3,1263022,BGF리테일,2020,2371169598247.000000,28.494404,1674341687928.000000,0.706125,2341969263000,1.693810,282330,3536000000.000000,34.120000,55.360000,122711267434.000000,5946068219555.000000,6181269419288.000000,NaN,3536000000.000000,3234000000.000000,6181269419290.000000
4,858364,BNK금융지주,2020,NaN,NaN,NaN,NaN,1851312197280,NaN,138930,NaN,51.700000,11.560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
for i in total_lst:
    try:
        print(i['advertising_expenses_x'].value_counts(dropna = False).iloc[0])
        print(i['current_revenue_x'].value_counts(dropna = False).iloc[0])
        print('\n')
        
    except:
        continue

203
50


200
51


200
52


226
30


290
28




In [27]:
import pandas as pd

# 처리해야 할 컬럼 쌍 정의 (확장성을 위해 리스트로 관리)
cols_to_fix = ['advertising_expenses', 'current_revenue', 'research_expenses']

for i, df in enumerate(total_lst):
    # 1. 필수 컬럼이 존재하는지 먼저 확인 (에러 방지)
    actual_cols = df.columns
    
    for col in cols_to_fix:
        col_x = f"{col}_x"
        col_y = f"{col}_y"
        
        if col_x in actual_cols and col_y in actual_cols:
            # fillna를 사용하여 null 값을 교체 (기존 mask 방식보다 간결하고 빠름)
            df[col_x] = df[col_x].fillna(df[col_y])
            
            # 사용이 끝난 _y 컬럼 제거
            df.drop(columns=[col_y], inplace=True)
            
            # _x 컬럼 이름 변경
            df.rename(columns={col_x: col}, inplace=True)
        else:
            # 컬럼이 없는 경우 로그 출력 (선택 사항)
            print(f"Index {i}: '{col_x}' 또는 '{col_y}'가 존재하지 않습니다.")

# 결과 확인
print("처리 완료")
if 'total_2020' in locals():
    print(total_2020.head())

Index 0: 'advertising_expenses_x' 또는 'advertising_expenses_y'가 존재하지 않습니다.
Index 0: 'current_revenue_x' 또는 'current_revenue_y'가 존재하지 않습니다.
Index 0: 'research_expenses_x' 또는 'research_expenses_y'가 존재하지 않습니다.
처리 완료
   corp_code corp_name  year          total_asset  asset_size  \
0     365387    AJ네트웍스  2020 1588170350000.000000   28.093604   
1     125080     AK홀딩스  2020 4152091609064.000000   29.054633   
2     219097       BGF  2020 1658204361732.000000   28.136756   
3    1263022    BGF리테일  2020 2371169598247.000000   28.494404   
4     858364   BNK금융지주  2020                  NaN         NaN   

            total_debt  debt_ratio     market_cap  tobins_q  stock_code  \
0 1300162539894.000000    0.818654   216084891425  0.954713       95570   
1 2908596226989.000000    0.700514   333176159150  0.780756        6840   
2  121105273379.000000    0.073034   468533691945  0.355589       27410   
3 1674341687928.000000    0.706125  2341969263000  1.693810      282330   
4                  NaN

In [ ]:
# for df in total_lst:
#     try:
#         mask_a = df['advertising_expenses_x'].isnull()
#         df.loc[mask_a, 'advertising_expenses_x'] = df.loc[mask_a, 'advertising_expenses_y']
        
#         mask_r = df['current_revenue_x'].isnull()
#         df.loc[mask_r, 'current_revenue_x'] = df.loc[mask_r, 'current_revenue_y']
        
#         df.drop(['advertising_expenses_y', 'current_revenue_y'], axis = 1, inplace = True)
        
#         df.rename(columns={
#             'advertising_expenses_x': 'advertising_expenses',
#             'current_revenue_x': 'current_revenue'
#         }, inplace=True)
    
#     except:
#         continue

# total_2020.head()

In [28]:
for i in total_lst:
    try:
        print(i['advertising_expenses'].value_counts(dropna = False).iloc[0])
        print(i['current_revenue'].value_counts(dropna = False).iloc[0])
        print('\n')
        
    except:
        continue

210
61


129
30


134
30


139
30


139
14


149
13




In [29]:
total_2024.head()

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q,stock_code,advertising_expenses,foreign_ownership_ratio,MSE,net_profit,previous_revenue,current_revenue,research_expenses
0,365387,AJ네트웍스,2024,1717770434389.000000,28.172048,1274047255964.000000,0.741687,194813127495,0.855097,95570,744191000.000000,7.950000,55.260000,21600800986.000000,936525061005.000000,1014156314426.000000,NaN
1,125080,AK홀딩스,2024,5336874928103.000000,29.305661,4091846427178.000000,0.766712,127706488040,0.790641,6840,105297623000.000000,12.750000,65.170000,-16585694879.000000,4479660141625.000000,4488280532801.000000,17281000000.000000
2,219097,BGF,2024,2300978258450.000000,28.464355,319875641992.000000,0.139017,330222928950,0.282531,27410,2596000000.000000,9.000000,69.720000,94776625368.000000,356392506416.000000,434085472661.000000,3282073000.000000
3,1263022,BGF리테일,2024,3402590175982.000000,28.855558,2218457219442.000000,0.651991,1771600365000,1.172653,282330,5128000000.000000,34.120000,53.390000,195217876198.000000,8194753978897.000000,8698757469111.000000,4535000000.000000
4,858364,BNK금융지주,2024,152470490926534.000000,32.657992,141322916379933.000000,0.926887,3313315757180,0.948618,138930,7611998000.000000,51.700000,10.420000,749976302023.000000,NaN,NaN,302120000000.000000


In [30]:
# 단위착각 이상치 확인
total_2024[total_2024['corp_name'] == '삼성출판사']

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q,stock_code,advertising_expenses,foreign_ownership_ratio,MSE,net_profit,previous_revenue,current_revenue,research_expenses
332,423690,삼성출판사,2024,208565174070.000000,26.063517,32179528826.000000,0.154290,148100000000,0.864380,68290,800186990000000.000000,1.410000,61.760000,10853322505.000000,41805582644.000000,40565887090.000000,840197140000000.000000


In [31]:
import pandas as pd
import numpy as np

# 1. 데이터프레임 리스트 설정
total_lst = [total_2019, total_2020, total_2021, total_2022, total_2023, total_2024]
year_labels = [2019, 2020, 2021, 2022, 2023, 2024]

for i, df in enumerate(total_lst):
    # --- [Step 1] 광고비(advertising_expenses) 보정 ---
    
    # 1차 보정 대상 (원본 > 매출 이지만, 1/1000 시 매출 이하인 경우)
    cond1_adv = (df['advertising_expenses'] > df['current_revenue']) & \
                (df['advertising_expenses'] / 1000 <= df['current_revenue'])
    df.loc[cond1_adv, 'advertising_expenses'] /= 1000
    
    # 2차 보정 대상 (1차 보정 후에도 > 매출 이지만, 1/1000000 시 매출 이하인 경우)
    cond2_adv = (df['advertising_expenses'] > df['current_revenue']) & \
                (df['advertising_expenses'] / 1000000 <= df['current_revenue'])
    df.loc[cond2_adv, 'advertising_expenses'] /= 1000000


    # --- [Step 2] 연구비(research_expenses) 보정 ---
    
    # 1차 보정 대상
    cond1_res = (df['research_expenses'] > df['current_revenue']) & \
                (df['research_expenses'] / 1000 <= df['current_revenue'])
    df.loc[cond1_res, 'research_expenses'] /= 1000
    
    # 2차 보정 대상
    cond2_res = (df['research_expenses'] > df['current_revenue']) & \
                (df['research_expenses'] / 1000000 <= df['current_revenue'])
    df.loc[cond2_res, 'research_expenses'] /= 1000000

    print(f"{year_labels[i]}년도 원본 데이터 보정 완료")

# 최종 확인: 여전히 이상치가 남아있는지 체크
print("\n--- 최종 잔여 이상치 확인 ---")
for i, df in enumerate(total_lst):
    remain = df[(df['advertising_expenses'] > df['current_revenue']) | 
               (df['research_expenses'] > df['current_revenue'])]
    print(f"{year_labels[i]}년도 잔여 이상치: {len(remain)}개")

2019년도 원본 데이터 보정 완료
2020년도 원본 데이터 보정 완료
2021년도 원본 데이터 보정 완료
2022년도 원본 데이터 보정 완료
2023년도 원본 데이터 보정 완료
2024년도 원본 데이터 보정 완료

--- 최종 잔여 이상치 확인 ---
2019년도 잔여 이상치: 0개
2020년도 잔여 이상치: 0개
2021년도 잔여 이상치: 0개
2022년도 잔여 이상치: 0개
2023년도 잔여 이상치: 0개
2024년도 잔여 이상치: 0개


In [32]:
total_2024[total_2024['corp_name'] == '삼성출판사']

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q,stock_code,advertising_expenses,foreign_ownership_ratio,MSE,net_profit,previous_revenue,current_revenue,research_expenses
332,423690,삼성출판사,2024,208565174070.000000,26.063517,32179528826.000000,0.154290,148100000000,0.864380,68290,800186990.000000,1.410000,61.760000,10853322505.000000,41805582644.000000,40565887090.000000,840197140.000000


In [33]:
total_lst = [total_2019, total_2020, total_2021, total_2022, total_2023, total_2024]

for df in total_lst:
    try:
        if df is total_2021:
            df.loc[df['corp_name'] == '하이트진로홀딩스', 'total_asset'] = 42363.0 * 100000000
            df.loc[df['corp_name'] == '하이트진로홀딩스', 'total_debt'] = 31489.2 * 100000000
            df.loc[df['corp_name'] == '하이트진로홀딩스', 'debt_ratio'] = ((31489.2 * 100000000) / (42363.0 * 100000000))
            df.loc[df['corp_name'] == '하이트진로홀딩스', 'tobins_q'] = ((df.loc[df['corp_name'] == '하이트진로홀딩스', 'market_cap'] + (31489.2 * 100000000)) / (42363.0 * 100000000))
            df.loc[df['corp_name'] == '하이트진로홀딩스', 'asset_size'] = np.log(42363.0 * 100000000)
        
        if df is total_2024:
            df.loc[df['corp_name'] == '선진', 'total_asset'] = 12842.3 * 100000000
            df.loc[df['corp_name'] == '선진', 'total_debt'] = 8208.8 * 100000000
            df.loc[df['corp_name'] == '선진', 'debt_ratio'] = ((8208.8 * 100000000) / (12842.3 * 100000000))
            df.loc[df['corp_name'] == '선진', 'tobins_q'] = ((df.loc[df['corp_name'] == '선진', 'market_cap'] + (8208.8 * 100000000)) / (12842.3 * 100000000))
            df.loc[df['corp_name'] == '선진', 'asset_size'] = np.log(12842.3 * 100000000)
        
        
        df.loc[:, 'roa'] = (df['net_profit'] / df['total_asset']) * 100
        # df.loc[:, 'roa'] = np.where(df['total_asset'] > 0, (df['net_profit'] / df['total_asset']) * 100, 0)
        
        df.loc[:, 'ar'] = (df['advertising_expenses'] / df['current_revenue']) * 100
        # df.loc[:, 'ar'] = np.where(df['current_revenue'] > 0, df['advertising_expenses'] / df['current_revenue'], 0)
        
        df.loc[:, 'sgr'] = ((df['current_revenue'] - df['previous_revenue']) / df['previous_revenue'])  * 100
        # df.loc[:, 'sgr'] = np.where(df['previous_revenue'] > 0, (df['current_revenue'] - df['previous_revenue']) / df['previous_revenue'], 0)
        
        df.loc[:, 'rd'] = (df['research_expenses'] / df['current_revenue']) * 100
        # df.loc[:, 'rd'] = np.where(df['current_revenue'] > 0, df['research_expenses'] / df['current_revenue'], 0)
              
        df.drop(['total_asset', 'total_debt', 'market_cap', 'advertising_expenses', 'net_profit', 'previous_revenue', 'current_revenue', 'research_expenses'], axis = 1, inplace = True)
        
    except:
        continue

In [34]:
total_2024

,corp_code,corp_name,year,asset_size,debt_ratio,tobins_q,stock_code,foreign_ownership_ratio,MSE,roa,ar,sgr,rd
0,365387,AJ네트웍스,2024,28.172048,0.741687,0.855097,95570,7.950000,55.260000,1.257491,0.073380,8.289287,NaN
1,125080,AK홀딩스,2024,29.305661,0.766712,0.790641,6840,12.750000,65.170000,-0.310775,2.346057,0.192434,0.385025
2,219097,BGF,2024,28.464355,0.139017,0.282531,27410,9.000000,69.720000,4.118971,0.598039,21.799832,0.756089
3,1263022,BGF리테일,2024,28.855558,0.651991,1.172653,282330,34.120000,53.390000,5.737331,0.058951,6.150319,0.052134
4,858364,BNK금융지주,2024,32.657992,0.926887,0.948618,138930,51.700000,10.420000,0.491883,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2024,26.208110,0.287235,0.738809,5870,24.830000,22.830000,4.457985,0.226521,0.855043,4.983916
727,362238,휴비스,2024,27.363636,0.692983,0.809279,79980,3.380000,51.000000,-17.323999,0.016087,1.266432,0.428967
728,156488,휴스틸,2024,27.989580,0.217727,0.366121,5010,4.200000,48.060000,1.558903,0.013310,-5.461709,0.046218
729,103176,흥국화재,2024,30.187837,0.940079,0.958364,540,2.170000,79.190000,0.827643,NaN,NaN,0.739149


In [35]:
# 없음
total_2024[(total_2024['ar'] +  total_2024['rd']) > 100]

,corp_code,corp_name,year,asset_size,debt_ratio,tobins_q,stock_code,foreign_ownership_ratio,MSE,roa,ar,sgr,rd


In [36]:
total_lst = [total_2019, total_2020, total_2021, total_2022, total_2023, total_2024]
fe_col = ['corp_code', 'corp_name', 'year', 'A_SIZE', 'LEV', 'TQ', 'stock_code', 'FOR', 'MSE', 'ROA', 'ADV', 'SGR', 'R&D']

for df in total_lst:
    df.columns = fe_col        

In [37]:
total_2019

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,ADV,SGR,R&D
0,365387,AJ네트웍스,2019,28.220615,0.807381,0.936429,95570,7.950000,50.050000,2.336252,0.408742,15.041958,NaN
1,125080,AK홀딩스,2019,29.096118,0.669605,0.775513,6840,12.750000,64.890000,1.325746,1.695634,1.303060,0.513448
2,219097,BGF,2019,28.136260,0.077910,0.401320,27410,9.000000,69.980000,1.056805,NaN,8.939779,NaN
3,1263022,BGF리테일,2019,28.406859,0.713408,2.061966,282330,34.120000,55.420000,6.968159,0.035267,2.946848,NaN
4,858364,BNK금융지주,2019,NaN,NaN,NaN,138930,51.700000,9.190000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,26.287653,0.556077,0.949323,5870,24.830000,22.830000,3.051838,0.326546,-25.499606,0.332867
727,362238,휴비스,2019,27.499968,0.548405,0.787931,79980,3.380000,51.000000,-0.033440,0.050015,-18.468584,0.036018
728,156488,휴스틸,2019,27.084735,0.315212,0.446562,5010,4.200000,49.570000,-2.165968,0.028970,-8.794608,NaN
729,103176,흥국화재,2019,NaN,NaN,NaN,540,2.170000,79.190000,NaN,NaN,NaN,NaN


In [38]:
ESG_df = []

for year in range(2020, 2026):
    ESG = pd.read_csv(f'data/krx_esg_2020_2025/krx_esg_{year}_clean.csv')
    ESG_df.append(ESG)
    
ESG_2020, ESG_2021, ESG_2022, ESG_2023, ESG_2024, ESG_2025 = ESG_df
ESG_2020

,year,isu_cd,company_name,kcgs_total,kcgs_env,kcgs_soc,kcgs_gov,rpt_yn,key1,key2,crawl_year,crawl_page
0,2020,95570,AJ네트웍스,B,D,B,B,N,N,N,2020,1
1,2020,6840,AK홀딩스,B+,C,B+,B+,Y,N,N,2020,1
2,2020,27410,BGF,B+,B,A,B+,N,N,N,2020,1
3,2020,282330,BGF리테일,A,B+,B+,A,Y,N,N,2020,1
4,2020,138930,BNK금융지주,A+,B+,A+,A+,Y,N,N,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...
735,2020,5870,휴니드테크놀러지스,C,C,B,B,N,N,N,2020,74
736,2020,79980,휴비스,B+,B+,B+,B+,N,N,N,2020,74
737,2020,5010,휴스틸,C,B,B,C,N,N,N,2020,74
738,2020,540,흥국화재,B,D,A,B,N,N,N,2020,74


In [39]:
ESG_lst = [ESG_2020, ESG_2021, ESG_2022, ESG_2023, ESG_2024, ESG_2025]

for df in ESG_lst:
    df.drop(['year', 'isu_cd', 'rpt_yn', 'key1', 'key2', 'crawl_year', 'crawl_page'], axis = 1, inplace = True)
    df.columns = ['corp_name', 'ESG', 'E', 'S', 'G']

In [40]:
def get_result_list(tdf_list, esg_list):
    results = []
    for tdf, esg in zip(tdf_list, esg_list):
        results.append(tdf.merge(esg, on='corp_name', how='left'))
    return results

res_2019, res_2020, res_2021, res_2022, res_2023, res_2024 = get_result_list(total_lst, ESG_lst)

In [41]:
res_2024

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,ADV,SGR,R&D,ESG,E,S,G
0,365387,AJ네트웍스,2024,28.172048,0.741687,0.855097,95570,7.950000,55.260000,1.257491,0.073380,8.289287,NaN,B+,C,A,A
1,125080,AK홀딩스,2024,29.305661,0.766712,0.790641,6840,12.750000,65.170000,-0.310775,2.346057,0.192434,0.385025,A,A,A+,B+
2,219097,BGF,2024,28.464355,0.139017,0.282531,27410,9.000000,69.720000,4.118971,0.598039,21.799832,0.756089,A,A,A+,B+
3,1263022,BGF리테일,2024,28.855558,0.651991,1.172653,282330,34.120000,53.390000,5.737331,0.058951,6.150319,0.052134,A+,A+,A+,A
4,858364,BNK금융지주,2024,32.657992,0.926887,0.948618,138930,51.700000,10.420000,0.491883,NaN,NaN,NaN,A,A+,A+,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2024,26.208110,0.287235,0.738809,5870,24.830000,22.830000,4.457985,0.226521,0.855043,4.983916,D,C,C,D
727,362238,휴비스,2024,27.363636,0.692983,0.809279,79980,3.380000,51.000000,-17.323999,0.016087,1.266432,0.428967,B+,B+,A,B+
728,156488,휴스틸,2024,27.989580,0.217727,0.366121,5010,4.200000,48.060000,1.558903,0.013310,-5.461709,0.046218,C,C,C,C
729,103176,흥국화재,2024,30.187837,0.940079,0.958364,540,2.170000,79.190000,0.827643,NaN,NaN,0.739149,B+,B+,B,B+


In [42]:
ROA_df = []

for year in range(2020, 2025):
    ROA = pd.read_csv(f'data/2020~2025_전체기업_ROA/{year}_ROA_final_robust.csv')
    ROA_df.append(ROA)
    
ROA_2020, ROA_2021, ROA_2022, ROA_2023, ROA_2024 = ROA_df
ROA_2024

,기업명,종목코드,ROA
0,AJ네트웍스,95570,1.293000
1,AK홀딩스,6840,-0.311000
2,BGF,27410,4.238000
3,BGF리테일,282330,5.896000
4,BNK금융지주,138930,0.503000
...,...,...,...
725,휴니드테크놀러지스,5870,3.977000
726,휴비스,79980,-17.130000
727,휴스틸,5010,1.618000
728,흥국화재,540,0.856000


In [43]:
ROA_lst = [ROA_2020, ROA_2021, ROA_2022, ROA_2023, ROA_2024]

for df in ROA_lst:
    df.columns = ['corp_name', 'stock_code', 'ROA']

for df in ROA_lst:
    df.drop('stock_code', axis = 1, inplace = True)
    
ROA_2020

,corp_name,ROA
0,AJ네트웍스,-0.196000
1,AK홀딩스,-6.234000
2,BGF,1.078000
3,BGF리테일,5.402000
4,BNK금융지주,0.514000
...,...,...
716,휴니드테크놀러지스,2.489000
717,휴비스,9.443000
718,휴스틸,1.461000
719,흥국화재,0.176000


In [44]:
res_lst = [res_2020, res_2021, res_2022, res_2023, res_2024]
ROA_lst = [ROA_2020, ROA_2021, ROA_2022, ROA_2023, ROA_2024]

fin_2020, fin_2021, fin_2022, fin_2023, fin_2024 = get_result_list(res_lst, ROA_lst)

In [45]:
fin_2020

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA_x,ADV,SGR,R&D,ESG,E,S,G,ROA_y
0,365387,AJ네트웍스,2020,28.093604,0.818654,0.954713,95570,7.950000,49.700000,-0.208845,0.326818,1.678662,NaN,B,D,B,B,-0.196000
1,125080,AK홀딩스,2020,29.054633,0.700514,0.780756,6840,12.750000,64.900000,-6.400992,2.497429,-30.311267,0.728670,B+,B,B+,B+,-6.234000
2,219097,BGF,2020,28.136756,0.073034,0.355589,27410,9.000000,69.720000,1.078332,2.965385,15.738760,0.001295,A,A,A+,A,1.078000
3,1263022,BGF리테일,2020,28.494404,0.706125,1.693810,282330,34.120000,55.360000,5.175137,0.057205,3.955575,0.052319,A,A,A+,A,5.402000
4,858364,BNK금융지주,2020,NaN,NaN,NaN,138930,51.700000,11.560000,NaN,NaN,NaN,NaN,A+,A,A+,A+,0.514000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2020,26.246607,0.523670,0.844911,5870,24.830000,22.830000,2.280875,0.062016,1.749543,3.908921,B,C,B+,B+,2.489000
727,362238,휴비스,2020,27.495040,0.454909,0.780598,79980,3.380000,51.000000,9.466678,0.049138,-10.593434,0.034621,A,B+,A+,A,9.443000
728,156488,휴스틸,2020,27.127055,0.313949,0.405919,5010,4.200000,49.670000,1.430358,0.031697,-31.914188,0.171879,C,C,C,B,1.461000
729,103176,흥국화재,2020,NaN,NaN,NaN,540,2.170000,79.190000,NaN,NaN,NaN,0.444184,B+,B,A,B,0.176000


In [50]:
fin_2019 = res_2019

In [46]:
fin_lst = [fin_2020, fin_2021, fin_2022, fin_2023, fin_2024]

for df in fin_lst:
    df['ROA_x'] = np.where(((df['ROA_x'].isna()) | (df['ROA_x'] == 0)), df['ROA_y'], df['ROA_x'])
    df.drop('ROA_y', axis = 1, inplace = True)
    df.rename(columns={'ROA_x':'ROA'}, inplace = True)

In [47]:
fin_2020#[res_2019.columns]

,corp_code,corp_name,year,A_SIZE,LEV,TQ,stock_code,FOR,MSE,ROA,ADV,SGR,R&D,ESG,E,S,G
0,365387,AJ네트웍스,2020,28.093604,0.818654,0.954713,95570,7.950000,49.700000,-0.208845,0.326818,1.678662,NaN,B,D,B,B
1,125080,AK홀딩스,2020,29.054633,0.700514,0.780756,6840,12.750000,64.900000,-6.400992,2.497429,-30.311267,0.728670,B+,B,B+,B+
2,219097,BGF,2020,28.136756,0.073034,0.355589,27410,9.000000,69.720000,1.078332,2.965385,15.738760,0.001295,A,A,A+,A
3,1263022,BGF리테일,2020,28.494404,0.706125,1.693810,282330,34.120000,55.360000,5.175137,0.057205,3.955575,0.052319,A,A,A+,A
4,858364,BNK금융지주,2020,NaN,NaN,NaN,138930,51.700000,11.560000,0.514000,NaN,NaN,NaN,A+,A,A+,A+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2020,26.246607,0.523670,0.844911,5870,24.830000,22.830000,2.280875,0.062016,1.749543,3.908921,B,C,B+,B+
727,362238,휴비스,2020,27.495040,0.454909,0.780598,79980,3.380000,51.000000,9.466678,0.049138,-10.593434,0.034621,A,B+,A+,A
728,156488,휴스틸,2020,27.127055,0.313949,0.405919,5010,4.200000,49.670000,1.430358,0.031697,-31.914188,0.171879,C,C,C,B
729,103176,흥국화재,2020,NaN,NaN,NaN,540,2.170000,79.190000,0.176000,NaN,NaN,0.444184,B+,B,A,B


In [48]:
import pandas as pd

# 1. 대상 연도와 기준 컬럼 정의
years = [2019, 2020, 2021, 2022, 2023, 2024]
col_lst = ['corp_code', 'corp_name', 'stock_code', 'year', 'TQ', 'ROA', 'ADV', 'SGR', 'LEV', 'R&D', 'A_SIZE', 'MSE', 'FOR', 'ESG', 'E', 'S', 'G']

# 2. globals()를 이용해 변수 자체를 강제로 교체
for year in years:
    var_name = f'fin_{year}'
    if var_name in globals():
        # 현재 변수에 담긴 데이터프레임을 가져와서 컬럼 순서 재배치
        df_temp = globals()[var_name]
        globals()[var_name] = df_temp[col_lst]

# 3. 변경 결과 확인 (이제는 확실히 바뀝니다)
print("--- 변경 후 fin_2024 컬럼 순서 ---")
print(fin_2024.columns.tolist())

--- 변경 후 fin_2024 컬럼 순서 ---
['corp_code', 'corp_name', 'stock_code', 'year', 'TQ', 'ROA', 'ADV', 'SGR', 'LEV', 'R&D', 'A_SIZE', 'MSE', 'FOR', 'ESG', 'E', 'S', 'G']


In [51]:
fin_lst = [fin_2019, fin_2020, fin_2021, fin_2022, fin_2023, fin_2024]

# 연산할 컬럼 리스트 (공통)
cols_to_multiply = ['TQ', 'ADV', 'SGR', 'LEV', 'R&D']

for df in fin_lst:
    # 1. 공통 컬럼 100 곱하기
    df[cols_to_multiply] = df[cols_to_multiply] * 100

# 결과 확인
print(fin_2019[['TQ', 'ROA', 'ADV']].head())

          TQ      ROA        ADV
0  93.642922 2.336252  40.874166
1  77.551265 1.325746 169.563436
2  40.132026 1.056805        NaN
3 206.196590 6.968159   3.526700
4        NaN      NaN        NaN


In [52]:
fin_2020

,corp_code,corp_name,stock_code,year,TQ,ROA,ADV,SGR,LEV,R&D,A_SIZE,MSE,FOR,ESG,E,S,G
0,365387,AJ네트웍스,95570,2020,95.471335,-0.208845,32.681754,167.866191,81.865433,NaN,28.093604,49.700000,7.950000,B,D,B,B
1,125080,AK홀딩스,6840,2020,78.075647,-6.400992,249.742912,-3031.126731,70.051350,72.867023,29.054633,64.900000,12.750000,B+,B,B+,B+
2,219097,BGF,27410,2020,35.558884,1.078332,296.538532,1573.876024,7.303399,0.129549,28.136756,69.720000,9.000000,A,A,A+,A
3,1263022,BGF리테일,282330,2020,169.381007,5.175137,5.720508,395.557520,70.612481,5.231935,28.494404,55.360000,34.120000,A,A,A+,A
4,858364,BNK금융지주,138930,2020,NaN,0.514000,NaN,NaN,NaN,NaN,NaN,11.560000,51.700000,A+,A,A+,A+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,5870,2020,84.491081,2.280875,6.201555,174.954329,52.367030,390.892104,26.246607,22.830000,24.830000,B,C,B+,B+
727,362238,휴비스,79980,2020,78.059771,9.466678,4.913826,-1059.343383,45.490942,3.462119,27.495040,51.000000,3.380000,A,B+,A+,A
728,156488,휴스틸,5010,2020,40.591900,1.430358,3.169662,-3191.418843,31.394881,17.187895,27.127055,49.670000,4.200000,C,C,C,B
729,103176,흥국화재,540,2020,NaN,0.176000,NaN,NaN,NaN,44.418389,NaN,79.190000,2.170000,B+,B,A,B


In [53]:
import os

# 폴더가 없는 경우를 대비해 폴더 생성 (선택 사항)
os.makedirs('data/fin', exist_ok=True)

fin_lst = [fin_2019, fin_2020, fin_2021, fin_2022, fin_2023, fin_2024]

for df in fin_lst:
    # 1. 해당 데이터프레임의 'year' 컬럼에서 첫 번째 행의 값을 가져옵니다.
    year = df['year'].iloc[0]
    
    # 2. 파일 경로 설정 및 저장
    # 한글 깨짐 방지를 위해 encoding='utf-8-sig'를 권장합니다.
    file_path = f'data/fin/fin_{year}.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    
    print(f"저장 완료: {file_path}")

저장 완료: data/fin/fin_2019.csv
저장 완료: data/fin/fin_2020.csv
저장 완료: data/fin/fin_2021.csv
저장 완료: data/fin/fin_2022.csv
저장 완료: data/fin/fin_2023.csv
저장 완료: data/fin/fin_2024.csv


In [54]:
# 전체 합치기
total_df = pd.concat(fin_lst, axis=0, ignore_index=True)
total_df.to_csv('data/fin/fin_total_all_years.csv', index=False, encoding='utf-8-sig')
print("통합 파일 저장 완료!")

통합 파일 저장 완료!


## 데이터 칼럼 전처리2

In [ ]:
import pandas as pd
from functools import reduce
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
import dart_fss as dart
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
pd.options.display.float_format = '{:.6f}'.format

In [ ]:
# 전처리 데이터프레임 생성
fin_df = []

for year in range(2019, 2025):
    fin = pd.read_csv(f'fin/fin_{year}.csv', index_col = 0).replace(0, np.nan)
    fin_df.append(fin)
    
fin_2019, fin_2020, fin_2021, fin_2022, fin_2023, fin_2024 = fin_df

fin_all = pd.read_csv(f'fin/fin_total_all_years.csv').replace(0, np.nan)

fin_all

In [ ]:
# 컬럼은 그대로 두되, 정렬만 기업코드 -> 연도순으로 수행
fin_all = fin_all.sort_values(by=['corp_code', 'year'], ascending=[True, True])

# 확인
fin_all.head(10)

In [ ]:
numeric_values = ['TQ', 'ROA', 'ADV', 'SGR', 'LEV', 'R&D', 'A_SIZE', 'MSE', 'FOR']

pd.pivot_table(data = fin_all, 
                           index = ['corp_code', 'corp_name', 'year'], 
                           values = numeric_values)

In [ ]:
# # 1. corp_code와 corp_name을 인덱스로 설정
# # 2. 인덱스를 기준으로 정렬 (기업코드순 -> 그 안에서 연도순)
# fin_all_indexed = fin_all.set_index(['corp_code', 'corp_name']).sort_index()

# # 결과 확인
# fin_all_indexed.head(10)